# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [3]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations

from tqdm import tqdm_notebook

%autosave 5
%matplotlib inline

ModuleNotFoundError: No module named 'keras'

In [2]:
import os
os.chdir(r'C:\Users\Saurabh\Downloads\Deployement\Deployement')

## Fetching Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [ ]:
print(CoinMarketCap())

Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [ ]:
historic_data = CoinMarketCap.historic()
historic_data.head()

The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [ ]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` variable.
#


## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [ ]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M.build()

In [ ]:
M.train(epochs=100, verbose=1)

We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [ ]:
M.predict(denormalized=True)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [ ]:
M.evaluate()

Finally, we can now save the trained model on disk for later use.

In [ ]:
M.save('bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [ ]:
M = Model(path='bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M.predict(denormalized=True)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [ ]:
M1 = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [ ]:
M1.build()

In [ ]:
M1.train()

In [ ]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
results = []
for i in range(A, B):
    M1.train(model_data[C:D])
    results.append(M.evaluate())

In [ ]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

In [ ]:
M1.predict(denormalized=True)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [ ]:
old_data = model_data[0*7:7*48 + 7]

In [ ]:
new_data = model_data[0*7:7*49 + 7]

In [ ]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M2.build()
M1.train()

In [ ]:
M2.predict(denormalized=True)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [ ]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#


In [ ]:
M3.build()
M3.train()

In [ ]:
M3.predict(denormalized=True)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.